# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Ramirez  ->  M. Ramirez-Tannus  |  ['M. Ramirez']
M. Prunier  ->  M. Prunier  |  ['M. Prunier']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Samland  ->  M. Samland  |  ['M. Samland']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
Arxiv has 75 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.10701


extracting tarball to tmp_2409.10701...

 done.
Retrieving document from  https://arxiv.org/e-print/2409.10711


extracting tarball to tmp_2409.10711...

...

✔ → 31661:\section{Acknowledgements}
  ↳ 35039:\section{Software \& Facilities}
✔ → 35039:\section{Software \& Facilities}
  ↳ 35365:\begin{appendix}
✔ → 35365:\begin{appendix}
  ↳ 35382:\section{Image De-convolution}\label{app:deconvolution}
✔ → 35382:\section{Image De-convolution}\label{app:deconvolution}
  ↳ 36244:\section{Fits to Atomic Lines}\label{app:atom}
✔ → 36244:\section{Fits to Atomic Lines}\label{app:atom}
  ↳ 37793:end


/tmp/ipykernel_2742/1211882699.py:51: LatexWarning: 2409.11176 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2409.11395


extracting tarball to tmp_2409.11395...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.10711-b31b1b.svg)](https://arxiv.org/abs/2409.10711) | **Deconvolving X-ray Galaxy Cluster Spectra Using a Recurrent Inference Machine**  |
|| C. Rhea, et al. -- incl., <mark>M. Prunier</mark> |
|*Appeared on*| *2024-09-18*|
|*Comments*| *Submitted to AJ*|
|**Abstract**|            Recent advances in machine learning algorithms have unlocked new insights in observational astronomy by allowing astronomers to probe new frontiers. In this article, we present a methodology to disentangle the intrinsic X-ray spectrum of galaxy clusters from the instrumental response function. Employing state-of-the-art modeling software and data mining techniques of the Chandra data archive, we construct a set of 100,000 mock Chandra spectra. We train a recurrent inference machine (RIM) to take in the instrumental response and mock observation and output the intrinsic X-ray spectrum. The RIM can recover the mock intrinsic spectrum below the 1-$\sigma$ error threshold; moreover, the RIM reconstruction of the mock observations are indistinguishable from the observations themselves. To further test the algorithm, we deconvolve extracted spectra from the central regions of the galaxy group NGC 1550, known to have a rich X-ray spectrum, and the massive galaxy clusters Abell 1795. Despite the RIM reconstructions consistently remaining below the 1-$\sigma$ noise level, the recovered intrinsic spectra did not align with modeled expectations. This discrepancy is likely attributable to the RIM's method of implicitly encoding prior information within the neural network. This approach holds promise for unlocking new possibilities in accurate spectral reconstructions and advancing our understanding of complex X-ray cosmic phenomena.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.10701-b31b1b.svg)](https://arxiv.org/abs/2409.10701) | **A Novel Optimal Transport-Based Approach for Interpolating Spectral Time Series: Paving the Way for Photometric Classification of Supernovae**  |
|| <mark>M. Ramirez</mark>, et al. |
|*Appeared on*| *2024-09-18*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            This paper introduces a novel method for creating spectral time series, which can be used for generating synthetic light curves for photometric classification but also for applications like K-corrections and bolometric corrections. This approach is particularly valuable in the era of large astronomical surveys, where it can significantly enhance the analysis and understanding of an increasing number of SNe, even in the absence of extensive spectroscopic data. methods: By employing interpolations based on optimal transport theory, starting from a spectroscopic sequence, we derive weighted average spectra with high cadence. The weights incorporate an uncertainty factor, for penalizing interpolations between spectra with significant epoch differences and with poor match between the synthetic and observed photometry. results: Our analysis reveals that even with phase difference of up to 40 days between pairs of spectra, optical transport can generate interpolated spectral time series that closely resemble the original ones. Synthetic photometry extracted from these spectral time series aligns well with observed photometry. The best results are achieved in the V band, with relative residuals less than 10% for 87% and 84% of the data for type Ia and II, respectively. For the B, g, R and r bands the relative residuals are between 65% and 87% within the previously mentioned 10% threshold for both classes. The worse results correspond to the i and I bands where, in the case, of SN~Ia the values drop to 53% and 42%, respectively. conclusions: We introduce a new method to construct spectral time series for individual SN starting from a sparse spectroscopic sequence, demonstrating its capability to produce reliable light curves that can be used for photometric classification.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.10799-b31b1b.svg)](https://arxiv.org/abs/2409.10799) | **Alpha-Proton Differential Flow of A Coronal Mass Ejection at 15 Solar Radii**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2024-09-18*|
|*Comments*| *8 pages, 3 figures*|
|**Abstract**|            Alpha-proton differential flow ($V_{\alpha p}$) of coronal mass ejections (CMEs) and solar wind from the Sun to 1 au and beyond could influence the instantaneous correspondence of absolute abundances of alpha particles (He$^{2+}$/H$^{+}$) between solar corona and interplanetary space as the abundance of a coronal source can vary with time. Previous studies based on Ulysses and Helios showed that $V_{\alpha p}$ is negligible within CMEs from 5 to 0.3 au, similar to slow solar wind ($<$ 400 km s$^{-1}$). However, recent new observations using Parker Solar Probe (PSP) revealed that the $V_{\alpha p}$ of slow wind increases to $\sim$60 km s$^{-1}$ inside 0.1 au. It is significant to answer whether the $V_{\alpha p}$ of CMEs exhibits the similar behavior near the Sun. In this Letter, we report the $V_{\alpha p}$ of a CME measured by PSP at $\sim$15 $R_\odot$ for the first time, which demonstrates that the $V_{\alpha p}$ of CMEs is obvious and complex inside 0.1 au while keeps lower than the local Alfvén speed. A very interesting point is that the same one CME duration can be divided into A and B intervals clearly with Coulomb number below and beyond 0.5, respectively. The means of $V_{\alpha p}$ and alpha-to-proton temperature ratios of interval A (B) is 96.52 (21.96) km s$^{-1}$ and 7.65 (2.23), respectively. This directly illustrates that Coulomb collisions play an important role in reducing the non-equilibrium features of CMEs. Our study indicates that the absolute elemental abundances of CMEs also might vary during their propagation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.10963-b31b1b.svg)](https://arxiv.org/abs/2409.10963) | **JWST PRIMER: A lack of outshining in four normal z =4-6 galaxies from the ALMA-CRISTAL Survey**  |
|| N. E. P. Lines, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-09-18*|
|*Comments*| *16 pages, 8 figures, 3 tables, plus 4 page appendix. Submitted to MNRAS*|
|**Abstract**|            We present a spatially resolved analysis of four star-forming galaxies at $z = 4.44-5.64$ using data from the JWST PRIMER and ALMA-CRISTAL surveys to probe the stellar and inter-stellar medium properties on the sub-kpc scale. In the $1-5\,\mu{\rm m}$ JWST NIRCam imaging we find that the galaxies are composed of multiple clumps (between $2$ and $\sim 8$) separated by $\simeq 5\,{\rm kpc}$, with comparable morphologies and sizes in the rest-frame UV and optical. Using BAGPIPES to perform pixel-by-pixel SED fitting to the JWST data we show that the SFR ($\simeq 25\,{\rm M}_{\odot}/{\rm yr}$) and stellar mass (${\rm log}_{10}(M_{\star}/{\rm M}_{\odot}) \simeq 9.5$) derived from the resolved analysis are in close ($ \lesssim 0.3\,{\rm dex}$) agreement with those obtained by fitting the integrated photometry. In contrast to studies of lower-mass sources, we thus find a reduced impact of outshining of the older (more massive) stellar populations in these normal $z \simeq 5$ galaxies. Our JWST analysis recovers bluer rest-frame UV slopes ($\beta \simeq -2.1$) and younger ages ($\simeq 100\,{\rm Myr}$) than archival values. We find that the dust continuum from ALMA-CRISTAL seen in two of these galaxies correlates, as expected, with regions of redder rest-frame UV slopes and the SED-derived $A_{\rm V}$, as well as the peak in the stellar mass map. We compute the resolved IRX-$\beta$ relation, showing that the IRX is consistent with the local starburst attenuation curve and further demonstrating the presence of an inhomogeneous dust distribution within the galaxies. A comparison of the CRISTAL sources to those from the FirstLight zoom-in simulation of galaxies with the same $M_{\star}$ and SFR reveals similar age and colour gradients, suggesting that major mergers may be important in the formation of clumpy galaxies at this epoch.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11083-b31b1b.svg)](https://arxiv.org/abs/2409.11083) | **Characterisation of TOI-406 as showcase of the THIRSTEE program: A 2-planet system straddling the M-dwarf density gap**  |
|| G. Lacedelli, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2024-09-18*|
|*Comments*| *24 pages, 21 figures. SUBMITTED to A&A*|
|**Abstract**|            The exoplanet sub-Neptune population currently poses a conundrum. Are small-size planets volatile-rich cores without atmosphere, or are they rocky cores surrounded by H-He envelope? To test the different hypotheses from an observational point of view, a large sample of small-size planets with precise mass and radius measurements is the first necessary step. On top of that, much more information will likely be needed, including atmospheric characterisation and a demographic perspective on their bulk properties. We present the concept and strategy of THIRSTEE, a project which aims at shedding light on the composition of the sub-Neptune population across stellar types by increasing their number and improving the accuracy of bulk density measurements, as well as investigating their atmospheres and performing statistical, demographic analysis. We report the first results of the program, characterising a 2-planet system around the M dwarf TOI-406. We analyse TESS and ground-based photometry, together with ESPRESSO and NIRPS/HARPS RVs to derive the orbital parameters and investigate the internal composition of the 2 planets orbiting TOI-406, which have radii and masses of $R_b = 1.32 \pm 0.12 R_{\oplus}$, $M_b = 2.08_{-0.22}^{+0.23} M_{\oplus}$ and $R_c = 2.08_{-0.15}^{+0.16} R_{\oplus}$, $M_c = 6.57_{-0.90}^{+1.00} M_{\oplus}$, and periods of $3.3$ and $13.2$ days, respectively. Planet b is consistent with an Earth-like composition, while planet c is compatible with multiple internal composition models, including volatile-rich planets without H/He atmospheres. The 2 planets are located in 2 distinct regions in the mass-density diagram, supporting the existence of a density gap among small exoplanets around M dwarfs. With an equilibrium temperature of only 368 K, TOI-406 c stands up as a particularly interesting target for atmospheric characterisation with JWST in the low-temperature regime.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11173-b31b1b.svg)](https://arxiv.org/abs/2409.11173) | **Exploring the Key Features of Repeating Fast Radio Bursts with Machine Learning**  |
|| W.-P. Sun, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-09-18*|
|*Comments*| *15 pages, 7 figures*|
|**Abstract**|            Fast radio bursts (FRBs) are enigmatic high-energy events with unknown origins, which are observationally divided into two categories, i.e., repeaters and non-repeaters. However, there are potentially a number of non-repeaters that may be misclassified, as repeating bursts are missed due to the limited sensitivity and observation periods, thus misleading the investigation of their physical properties. In this work, we propose a repeater identification method based on the t-distributed Stochastic Neighbor Embedding (t-SNE) algorithm and apply the classification to the first Canadian Hydrogen Intensity Mapping Experiment Fast Radio Burst (CHIME/FRB) catalog. We find that the spectral morphology parameters, specifically spectral running ($r$), represent the key features for identifying repeaters from the non-repeaters. Also, the results suggest that repeaters are more biased towards narrowband emission, whereas non-repeaters are inclined toward broadband emission. We provide a list of 163 repeater candidates, with $5$ of which are confirmed with an updated repeater catalog from CHIME/FRB. Our findings help to the understanding of the various properties underlying repeaters and non-repeaters, as well as guidelines for future FRB detection and categorization.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11395-b31b1b.svg)](https://arxiv.org/abs/2409.11395) | **Quartz Clouds in the Dayside Atmosphere of the Quintessential Hot Jupiter HD 189733 b**  |
|| J. Inglis, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2024-09-18*|
|*Comments*| *21 pages, 7 Figures, 3 Tables, Accepted to ApJL*|
|**Abstract**|            Recent mid-infrared observations with JWST/MIRI have resulted in the first direct detections of absorption features from silicate clouds in the transmission spectra of two transiting exoplanets, WASP-17 b and WASP-107 b. In this paper, we measure the mid-infrared ($5-12$ $\mu$m) dayside emission spectrum of the benchmark hot Jupiter HD 189733 b with MIRI LRS by combining data from two secondary eclipse observations. We confirm the previous detection of H$_2$O absorption at 6.5 $\mu$m from Spitzer/IRS and additionally detect H$_2$S as well as an absorption feature at 8.7 $\mu$m in both secondary eclipse observations. The excess absorption at 8.7 $\mu$m can be explained by the presence of small ($\sim$0.01 $\mu$m) grains of SiO$_2$[s] in the uppermost layers of HD 189733 b's dayside atmosphere. This is the first direct detection of silicate clouds in HD 189733 b's atmosphere, and the first detection of a distinct absorption feature from silicate clouds on the day side of any hot Jupiter. We find that models including SiO$_2$[s] are preferred by $6-7\sigma$ over clear models and those with other potential cloud species. The high altitude location of these silicate particles is best explained by formation in the hottest regions of HD 189733 b's dayside atmosphere near the substellar point. We additionally find that HD 189733 b's emission spectrum longward of 9 $\mu$m displays residual features not well captured by our current atmospheric models. When combined with other JWST observations of HD 189733 b's transmission and emission spectrum at shorter wavelengths, these observations will provide us with the most detailed picture to date of the atmospheric composition and cloud properties of this benchmark hot Jupiter.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.11176-b31b1b.svg)](https://arxiv.org/abs/2409.11176) | **MINDS. JWST-MIRI Observations of a Spatially Resolved Atomic Jet and Polychromatic Molecular Wind Toward SY Cha**  |
|| K. R. Schwarz, et al. -- incl., <mark>M. Samland</mark>, <mark>T. Henning</mark>, <mark>G. Perotti</mark> |
|*Appeared on*| *2024-09-18*|
|*Comments*| *16 pages, 13 figures, 4 tables, submitted to ApJ Letters*|
|**Abstract**|            The removal of angular momentum from protostellar systems drives accretion onto the central star and may drive the dispersal of the protoplanetary disk. Winds and jets can contribute to removing angular momentum from the disk, though the dominant process remain unclear. To date, observational studies of resolved disk winds have mostly targeted highly inclined disks. We report the detection of extended H2 and [Ne II] emission toward the young stellar object SY Cha with the JWST Mid-InfraRed Instrument Medium Resolution Spectrometer (MIRI-MRS). This is one of the first polychromatic detections of extended H2 toward a moderately inclined, i=51.1 degrees, Class II source. We measure the semi-opening angle of the H2 emission as well as build a rotation diagram to determine the H2 excitation temperature and abundance. We find a wide semi-opening angle, high temperature, and low column density for the H2 emission, all of which are characteristic of a disk wind. These observations demonstrate MIRI-MRS's utility in expanding studies of resolved disk winds beyond edge-on sources.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.10711/./ModelComparison.png', 'tmp_2409.10711/./ObsID5289.png', 'tmp_2409.10711/./RIM.png']
copying  tmp_2409.10711/./ModelComparison.png to _build/html/
copying  tmp_2409.10711/./ObsID5289.png to _build/html/
copying  tmp_2409.10711/./RIM.png to _build/html/
exported in  _build/html/2409.10711.md
    + _build/html/tmp_2409.10711/./ModelComparison.png
    + _build/html/tmp_2409.10711/./ObsID5289.png
    + _build/html/tmp_2409.10711/./RIM.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$</div>



<div id="title">

# Deconvolving X-ray Galaxy Cluster Spectra Using a Recurrent Inference Machine

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.10711-b31b1b.svg)](https://arxiv.org/abs/2409.10711)<mark>Appeared on: 2024-09-18</mark> -  _Submitted to AJ_

</div>
<div id="authors">

C. L. Rhea, et al. -- incl., <mark>M. Prunier</mark>

</div>
<div id="abstract">

**Abstract:** Recent advances in machine learning algorithms have unlocked new insights in observational astronomy by allowing astronomers to probe new frontiers. In this article, we present a methodology to disentangle the intrinsic X-ray spectrum of galaxy clusters from the instrumental response function. Employing state-of-the-art modeling software and data mining techniques of the Chandra data archive, we construct a set of 100,000 mock Chandra spectra. We train a recurrent inference machine (RIM) to take in the instrumental response and mock observation and output the intrinsic X-ray spectrum. The RIM can recover the mock intrinsic spectrum below the 1- $\sigma$ error threshold; moreover, the RIM reconstruction of the mock observations are indistinguishable from the observations themselves. To further test the algorithm, we deconvolve extracted spectra from the central regions of the galaxy group NGC 1550, known to have a rich X-ray spectrum, and the massive galaxy clusters Abell 1795.  Despite the RIM reconstructions consistently remaining below the 1- $\sigma$ noise level, the recovered intrinsic spectra did not align with modeled expectations. This discrepancy is likely attributable to the RIM's method of implicitly encoding prior information within the neural network. This approach holds promise for unlocking new possibilities in accurate spectral reconstructions and advancing our understanding of  complex X-ray cosmic phenomena.

</div>

<div id="div_fig1">

<img src="tmp_2409.10711/./ModelComparison.png" alt="Fig2" width="100%"/>

**Figure 2. -** In this figure we compare the RIM solution for both ObIDs (ObsID 3186 and ObsID 3187 in red) and the model taken from literature values (green) for NGC 1550. The later was obtained by estimating the thermodynamic properties of the cluster from the convolved observed spectrum in Kolokythas2020 and then creating a mock spectrum using in \texttt{SOXS}. (*fig:modelComparison*)

</div>
<div id="div_fig2">

<img src="tmp_2409.10711/./ObsID5289.png" alt="Fig8" width="100%"/>

**Figure 8. -** Results of the RIM on the massive galaxy cluster Abell 1795. In the left panels, we show the RIM solution (i.e. the result of the deconvolution process) for ObsID 5289 (red). The middle panel shows the observed spectrum after background subtraction (black) and the RIM reconstruction (blue). The RIM reconstruction results from passing the RIM solution through the forward model developed in equation \ref{eqn:spec_mat}. The right panel shows the residual between the observed spectrum and the RIM reconstruction normalized to the noise level of the observation. (*fig:realObs5289*)

</div>
<div id="div_fig3">

<img src="tmp_2409.10711/./RIM.png" alt="Fig1" width="100%"/>

**Figure 1. -** Schematic view of the RIM. Nodes in dark gray are treated as inputs to the RIM. Note that we separate the hidden layers before applying the RIM and combine them after applying the RIM. The teal box represents the RIM itself. (*fig:RIM*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.10711"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

181  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
